In [1]:
from ollama_qw import OllamaQW
qw = OllamaQW()

2025-01-15 08:05:39.820 | INFO     | ollama_qw:__init__:12 - [{'type': 'function', 'function': {'name': 'close_light', 'description': '关掉机箱的灯。', 'parameters': {'type': 'object', 'properties': {}, 'required': []}}}, {'type': 'function', 'function': {'name': 'open_light', 'description': '打开机箱的灯。', 'parameters': {'type': 'object', 'properties': {}, 'required': []}}}, {'type': 'function', 'function': {'name': 'run_software', 'description': '打开或者运行程序、软件、应用、脚本等。', 'parameters': {'type': 'object', 'properties': {'software_name': {'type': 'string', 'description': '需要打开或者运行的程序的简称或者别称，从enum中选择最相符的传入需要打开、运行或者执行的软件程序或者脚本的名称', 'enum': ['administrative tools', 'file explorer', 'ollama', 'sonic suite companion', 'wps office', 'livecaptions', 'magnify', 'narrator', 'on-screen keyboard', 'voiceaccess', 'blender 4', 'directx修复工具', 'gw-soul', 'unwise', 'kook', 'letsvpn', 'overwolf', 'valorant tracker', 'postman', 'raspberry pi imager', 'openrgb', 'sakuralauncher_8a49d589b8997c26e5fe8ca0c8308d23', 'steam'

In [2]:

def main():
    while True:
        msg = input(">: ")
        if msg.lower() == '/exit':
            break
        res = qw.chat(msg)
        print(f"Alice: {res['content']}")
main()

2025-01-15 08:05:56.678 | INFO     | ollama_qw:chat:27 - input message: {'role': 'user', 'content': 'Alice帮我打开cpuz'}
2025-01-15 08:05:58.360 | INFO     | ollama_qw:ollama_chat:21 - model response is: model='qwen2.5:3b' created_at='2025-01-15T00:05:58.3588261Z' done=True done_reason='stop' total_duration=1679285600 load_duration=1068546000 prompt_eval_count=280 prompt_eval_duration=159000000 eval_count=23 eval_duration=235000000 message=Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='run_software', arguments={'software_name': 'cpuz'}))])
2025-01-15 08:05:58.360 | INFO     | ollama_qw:chat:35 - function name: run_software
2025-01-15 08:05:58.361 | INFO     | ollama_qw:chat:36 - function args: {'software_name': 'cpuz'}


TypeError: unhashable type: 'list'

In [6]:
StatMenuSoftware.keys()

dict_keys(['Administrative Tools', 'File Explorer', 'Ollama', 'Sonic Suite Companion', 'WPS Office', 'LiveCaptions', 'Magnify', 'Narrator', 'On-Screen Keyboard', 'VoiceAccess', 'Blender 4', 'DirectX修复工具', 'GW-SOUL', 'Unwise', 'KOOK', 'LetsVPN', 'Overwolf', 'Valorant Tracker', 'Postman', 'Raspberry Pi Imager', 'OpenRGB', 'SakuraLauncher_8a49d589b8997c26e5fe8ca0c8308d23', 'Steam', 'Command Prompt', 'Control Panel', 'Run', 'Ubisoft Connect', 'Uplay', 'Install RELEASE (Ubuntu-22', 'WeMod', 'Windows PowerShell (x86)', 'Windows PowerShell', '小K直播姬', '百度网盘', '360安全卫士', '360安全防护中心', '7-Zip File Manager', 'Remote Desktop Connection', 'Steps Recorder', 'Windows Media Player Legacy', 'Character Map', 'Component Services', 'Computer Management', 'dfrgui', 'Disk Cleanup', 'Event Viewer', 'Hyper-V Manager', 'iSCSI Initiator', 'Memory Diagnostics Tool', 'ODBC Data Sources (32-bit)', 'ODBC Data Sources (64-bit)', 'Performance Monitor', 'Print Management', 'RecoveryDrive', 'Registry Editor', 'Resource 

In [21]:
from tools.tool_config import StatMenuSoftware
import difflib
key = difflib.get_close_matches('CPUZ',StatMenuSoftware.keys(),1, cutoff=0.5)
"三角洲".lower()

'三角洲'